In [ ]:
# Install the ultralytics package using pip
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
#Instance
model = YOLO('yolov8l-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8l-seg.pt')  # Transfer the weights from a pretrained model (recommended for training)

In [ ]:
!pip install roboflow

In [ ]:


from roboflow import Roboflow
rf = Roboflow(api_key="l3ZRwINmzPuMzyjtYwHf")
project = rf.workspace("md-hasibul-islam-dhrubo").project("pavement-damage-detection-3")
version = project.version(2)
dataset = version.download("yolov8")


In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat /kaggle/working/Pavement-damage-detection--3-2/data.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /kaggle/working/Pavement-damage-detection--3-2/data.yaml

names:
- Crack
- Edgecrack
- pothole
nc: 3
roboflow:
  license: CC BY 4.0
  project: pavement-damage-detection-3
  url: https://universe.roboflow.com/md-hasibul-islam-dhrubo/pavement-damage-detection-3/dataset/2
  version: 2
  workspace: md-hasibul-islam-dhrubo
test: /kaggle/working/Pavement-damage-detection--3-2/test/images
train: /kaggle/working/Pavement-damage-detection--3-2/train/images
val: /kaggle/working/Pavement-damage-detection--3-2/valid/images

In [ ]:
%cat /kaggle/working/Pavement-damage-detection--3-2/data.yaml

In [ ]:
# define number of classes based on YAML
import yaml
with open("/kaggle/working/Pavement-damage-detection--3-2/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

print(num_classes)

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key='cded8b1a7da321c343539af568490414199ad5c7')

In [ ]:
# Train the model
results = model.train(data='/kaggle/working/Pavement-damage-detection--3-2/data.yaml',
                      epochs=120,
                      patience=0, #I am setting patience=0 to disable early stopping.
                      batch=8,
                      imgsz=640)

In [ ]:
from IPython.display import Image

In [ ]:
Image("/kaggle/working/runs/segment/train/results.png")

In [ ]:
Image("/kaggle/working/runs/segment/train/train_batch1.jpg")

In [ ]:
%ls /kaggle/working/runs/segment/train/weights

In [ ]:
my_new_model = YOLO('/kaggle/working/runs/segment/train/weights/best.pt')

In [ ]:
new_image = '/kaggle/working/Pavement-damage-detection--3-2/test/images/IMG_20240308_134605_jpg.rf.9c12a479848ba77d29fb3fa0b71d956b.jpg'
new_results = my_new_model.predict(new_image, conf=0.2)  #Adjust conf threshold

In [ ]:
new_result_array = new_results[0].plot()
plt.figure(figsize=(8,8))
plt.imshow(new_result_array)